In [ ]:
import cv2
import numpy as np
from skimage.transform import rotate
from google.colab.patches import cv2_imshow
import argparse
from scipy import ndimage, misc

class preprocessing:

 def __init__(self):
   pass

 def findScore(self,img, angle):

  data = ndimage.rotate(img, angle, reshape = False, order = 0)
  hist = np.sum(data, axis = 1)
  score = np.sum((hist[1:] - hist[:-1]) ** 2)
  return hist, score


 def skewCorrect(self,img):
  #self.findScore()
  img = cv2.resize(img, (0, 0), fx = 0.75, fy = 0.75)
  delta = 1
  limit = 45
  angles = np.arange(-limit, limit+delta, delta)
  scores = []
  for angle in angles:
    hist, score = self.findScore(img, angle)
    scores.append(score)
   
        
  bestScore = max(scores)
  bestAngle = angles[scores.index(bestScore)]
  rotated = ndimage.rotate(img, bestAngle, reshape = False, order = 0)
 # print("[INFO] angle: {:.3f}".format(bestAngle))

  return rotated




 def preprocess_image(self,image):

  #image1 = cv2.imread(image)

  # Convert the image to a NumPy array
  image_array = np.array(image)

  # Convert the image to grayscale
  gray = cv2.cvtColor(image_array, cv2.COLOR_BGR2GRAY)

  # Skew Correction
  rotated_img = self.skewCorrect(gray)

  # Normalize image
  norm_img = np.zeros((800,800))
  final_img = cv2.normalize(rotated_img,  norm_img, 0, 255, cv2.NORM_MINMAX)

  # Apply median blur to reduce noise and smooth the image
  blur = cv2.medianBlur(final_img, 1)

  # # Apply mean blur to reduce noise
  den= cv2.fastNlMeansDenoising(blur)

  kernel = np.array([[0, -1, 0],
                   [-1, 5,-1],
                   [0, -1, 0]])
  
  kernel1 = np.ones((1,),np.uint8)


  # Sharpen the image to enhance text
  image_sharp = cv2.filter2D(src=den, ddepth=-1, kernel=kernel)


  # Opening
  opening = cv2.morphologyEx(image_sharp, cv2.MORPH_OPEN, kernel1)


  # Compute the threshold using Otsu's method
  threshold_otsu = cv2.threshold(opening, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
  dim = (900, 1300)
    # resize image
  resized = cv2.resize(threshold_otsu, dim, interpolation = cv2.INTER_AREA)
 # cv2_imshow(resized)
    #self.cropimage(img)
  return resized


In [ ]:
class cropm:

  def __init__(self):
   pass

  def cropimage(self,image):
   # image=self.resizeimage(img)
    original = image.copy()
    #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(image, (3, 3), 0)
    thresh = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

# Remove horizontal lines
    horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25,3))
    detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=1)
    cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = cnts[0] if len(cnts) == 2 else cnts[1]
    for c in cnts:
      cv2.drawContours(thresh, [c], -1, 0, -1)    
# Dilate to merge into a single contour
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,25))
    dilate = cv2.dilate(thresh, vertical_kernel, iterations=3)

# Find contours, sort for largest contour and extract ROI
    cnts, _ = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2:]
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)[:-1]
    for c in cnts:
       x,y,w,h = cv2.boundingRect(c)
       cv2.rectangle(image, (x, y), (x + w, y + h), (36,255,12), 4)
       ROI = original[y:y+h, x:x+w]
       break
 #   cv2_imshow(image)
#    cv2_imshow(ROI) 
    return ROI

 


In [ ]:
import cv2
import numpy as np
from skimage.transform import rotate
from google.colab.patches import cv2_imshow
import argparse
from scipy import ndimage, misc
import glob 
import os

path = "/content/drive/MyDrive/Books/booksAsImage/aub_aco000024_hi.pdf_dir/*.jpg"
os.mkdir("/content/drive/MyDrive/Books/preprocessed/aub_aco000024_hi.pdf_dir")
j=0;
for file in glob.glob(path):
  try:
     image= cv2.imread(file)
     img_u8 = image.astype(np.uint8)
     preprocessed = preprocessing()
     p=preprocessed.preprocess_image(img_u8)
     c = cropm()
     c1=c.cropimage(p)
     cv2.imwrite(f'/content/drive/MyDrive/Books/preprocessed/aub_aco000024_hi.pdf_dir/image{j}.jpg', c1)
     j=j+1  
  except:
    pass
